In [ ]:
# ======================================================
# Estimating Missing Exoplanet Properties Using ML
## In this notebook, we merged the four exoplanet data warehouse tables—planet, star, system, and fact—into a single 
## comprehensive dataset for analysis. We performed data cleaning, handled missing values, and implemented model-based 
## imputation using Random Forest regressors to estimate missing planet mass (pl_masse) and radius (pl_rade). The imputed 
## values were added as new columns while preserving the original measurements, creating a fully enriched dataset ready 
## for exploration, visualization, and predictive modeling. This setup allows us to analyze planetary characteristics, 
## study feature importance, and build ML-driven insights about exoplanets.
# ======================================================

In [6]:
# Load dimension and fact tables
planet = pd.read_csv("planet_dim.csv")
star = pd.read_csv("star_dim.csv")
system = pd.read_csv("system_dim.csv")
fact = pd.read_csv("exoplanet_fact.csv")

# Merge fact table with dimensions. Star Schema
df = (
    fact
    .merge(
        planet[['planet_id','pl_name','discoverymethod','disc_year']],
        on='planet_id',
        how='left'
    )
    .merge(
        star[['star_id','st_spectype','st_mass','st_rad','st_teff']],
        on='star_id',
        how='left'
    )
    .merge(
        system[['system_id','sy_dist','sy_plx']],
        on='system_id',
        how='left'
    )
)

df.head()

,rowid,planet_id,star_id,system_id,pl_orbper,pl_rade,pl_masse,pl_name,discoverymethod,disc_year,st_spectype,st_mass,st_rad,st_teff,sy_dist,sy_plx
0,1.0,3,22541,1,323.21,NaN,NaN,11 Com b,Radial Velocity,2007.0,K0 III,1.74,10.86,4841.0,93.18,10.71
1,2.0,3,2,1,326.03,NaN,NaN,11 Com b,Radial Velocity,2007.0,G8 III,2.70,19.00,4742.0,93.18,10.71
2,3.0,3,22539,1,NaN,NaN,NaN,11 Com b,Radial Velocity,2007.0,NaN,1.40,NaN,NaN,93.18,10.71
3,4.0,6,2227,2,516.22,NaN,NaN,11 UMi b,Radial Velocity,2009.0,K1 III,1.13,10.22,4580.0,125.32,7.95
4,5.0,6,22437,2,516.22,NaN,NaN,11 UMi b,Radial Velocity,2009.0,NaN,1.10,1.45,6397.0,125.32,7.95


In [5]:
# Import libraries
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

In [7]:
# ======================================================
## Predict missing pl_rade and pl_masse values using ML
# ======================================================

# Defining features used to predict planet mass
mass_features = [
    'pl_orbper',   # orbital period (days)
    'pl_rade',     # planet radius (Earth radii)
    'st_mass',     # stellar mass (solar masses)
    'st_rad',      # stellar radius (solar radii)
    'st_teff',     # stellar effective temperature (K)
    'sy_dist'      # system distance (parsecs)
]

target_mass = 'pl_masse'

In [ ]:
## Split Data: Known vs Missing Targets

# Rows with known planet mass (training data)
mass_train = df[df[target_mass].notna()].copy()

# Rows where planet mass is missing
mass_predict = df[df[target_mass].isna()].copy()

In [8]:
# ======================================================
## Preprocessing
# ======================================================

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# Numeric features only (physics-based)
numeric_features = [
    'pl_orbper',   # orbital period (days)
    'pl_rade',     # planet radius (Earth radii)
    'st_mass',     # stellar mass (solar masses)
    'st_rad',      # stellar radius (solar radii)
    'st_teff',     # stellar effective temperature (K)
    'sy_dist'      # system distance (parsecs)
]

# Simple preprocessing pipeline
preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

In [9]:
## ML Pipeline (RandomForest)
# Model-based imputation

X = df[numeric_features]
y = df['pl_masse']   

model = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('regressor', RandomForestRegressor(
        n_estimators=200,
        random_state=42,
        n_jobs=-1
    ))
])

In [11]:
# Keep only rows where planet mass is known
mass_train = df[df['pl_masse'].notna()].copy()

X = mass_train[numeric_features]
y = mass_train['pl_masse']

# Train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

model.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 Pipeline(steps=[('imputer',
                                  SimpleImputer(strategy='median'))])),
                ('regressor',
                 RandomForestRegressor(n_estimators=200, n_jobs=-1,
                                       random_state=42))])

In [12]:
from sklearn.metrics import r2_score, mean_absolute_error

y_pred = model.predict(X_test)

print("R²:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))


R²: 0.5903691370570388
MAE: 456.50597506995155


In [13]:
# Fit on all rows with known planet mass
model.fit(X, y)

Pipeline(steps=[('preprocess',
                 Pipeline(steps=[('imputer',
                                  SimpleImputer(strategy='median'))])),
                ('regressor',
                 RandomForestRegressor(n_estimators=200, n_jobs=-1,
                                       random_state=42))])

In [14]:
## Predict missing planet masses

missing_mask = df['pl_masse'].isna()
df.loc[missing_mask, 'pl_masse_ml'] = model.predict(df.loc[missing_mask, numeric_features])

# Tag the source
df['pl_masse_source'] = df['pl_masse'].notna().map({True: 'Observed', False: 'ML Estimated'})

In [15]:
## Check Results
df[df['pl_masse'].isna()].head()

,rowid,planet_id,star_id,system_id,pl_orbper,pl_rade,pl_masse,pl_name,discoverymethod,disc_year,st_spectype,st_mass,st_rad,st_teff,sy_dist,sy_plx,pl_masse_ml,pl_masse_source
0,1.0,3,22541,1,323.21,NaN,NaN,11 Com b,Radial Velocity,2007.0,K0 III,1.74,10.86,4841.0,93.18,10.71,1543.67100,ML Estimated
1,2.0,3,2,1,326.03,NaN,NaN,11 Com b,Radial Velocity,2007.0,G8 III,2.70,19.00,4742.0,93.18,10.71,3690.57760,ML Estimated
2,3.0,3,22539,1,NaN,NaN,NaN,11 Com b,Radial Velocity,2007.0,NaN,1.40,NaN,NaN,93.18,10.71,2767.43160,ML Estimated
3,4.0,6,2227,2,516.22,NaN,NaN,11 UMi b,Radial Velocity,2009.0,K1 III,1.13,10.22,4580.0,125.32,7.95,1701.47485,ML Estimated
4,5.0,6,22437,2,516.22,NaN,NaN,11 UMi b,Radial Velocity,2009.0,NaN,1.10,1.45,6397.0,125.32,7.95,1896.82995,ML Estimated


In [ ]:
## Defining features used to predict planet radius
mass_features = [
    'pl_orbper',   # orbital period (days)
    'pl_masse',     # planet mass (Earth mass)
    'st_mass',     # stellar mass (solar masses)
    'st_rad',      # stellar radius (solar radii)
    'st_teff',     # stellar effective temperature (K)
    'sy_dist'      # system distance (parsecs)
]

target_mass = 'pl_rade'

In [16]:
## Split Data: Known vs Missing Targets

target_radius = 'pl_rade'
# Rows with known planet radius 
rade_train = df[df[target_radius].notna()].copy()

# Rows where planet radius is missing 
rade_predict = df[df[target_radius].isna()].copy()

In [17]:
# ======================================================
## Preprocessing
# ======================================================

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# Numeric features only (physics-based)
numeric_features = [
    'pl_orbper',   # orbital period (days)
    'pl_masse',     # planet mass (Earth mass)
    'st_mass',     # stellar mass (solar masses)
    'st_rad',      # stellar radius (solar radii)
    'st_teff',     # stellar effective temperature (K)
    'sy_dist'      # system distance (parsecs)
]

# Simple preprocessing pipeline
preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

In [19]:
## ML Pipeline (RandomForest)
# Model-based imputation

X = df[numeric_features]
y = df['pl_rade']   

model = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('regressor', RandomForestRegressor(
        n_estimators=200,
        random_state=42,
        n_jobs=-1
    ))
])

In [20]:
# Keep only rows where planet radius is known
mass_train = df[df['pl_rade'].notna()].copy()

X = mass_train[numeric_features]
y = mass_train['pl_rade']

# Train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

model.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 Pipeline(steps=[('imputer',
                                  SimpleImputer(strategy='median'))])),
                ('regressor',
                 RandomForestRegressor(n_estimators=200, n_jobs=-1,
                                       random_state=42))])

In [21]:
## Predict missing planet masses

missing_mask = df['pl_rade'].isna()
df.loc[missing_mask, 'pl_rade_ml'] = model.predict(df.loc[missing_mask, numeric_features])

# Tag the source
df['pl_rade_source'] = df['pl_rade'].notna().map({True: 'Observed', False: 'ML Estimated'})

## Check Results
df[df['pl_rade'].isna()].head()

,rowid,planet_id,star_id,system_id,pl_orbper,pl_rade,pl_masse,pl_name,discoverymethod,disc_year,st_spectype,st_mass,st_rad,st_teff,sy_dist,sy_plx,pl_masse_ml,pl_masse_source,pl_rade_ml,pl_rade_source
0,1.0,3,22541,1,323.21,NaN,NaN,11 Com b,Radial Velocity,2007.0,K0 III,1.74,10.86,4841.0,93.18,10.71,1543.67100,ML Estimated,7.086900,ML Estimated
1,2.0,3,2,1,326.03,NaN,NaN,11 Com b,Radial Velocity,2007.0,G8 III,2.70,19.00,4742.0,93.18,10.71,3690.57760,ML Estimated,7.221600,ML Estimated
2,3.0,3,22539,1,NaN,NaN,NaN,11 Com b,Radial Velocity,2007.0,NaN,1.40,NaN,NaN,93.18,10.71,2767.43160,ML Estimated,4.883455,ML Estimated
3,4.0,6,2227,2,516.22,NaN,NaN,11 UMi b,Radial Velocity,2009.0,K1 III,1.13,10.22,4580.0,125.32,7.95,1701.47485,ML Estimated,5.875700,ML Estimated
4,5.0,6,22437,2,516.22,NaN,NaN,11 UMi b,Radial Velocity,2009.0,NaN,1.10,1.45,6397.0,125.32,7.95,1896.82995,ML Estimated,8.521050,ML Estimated


In [24]:
## Fill NaNs only with ML values in a separate column
df['pl_masse_filled'] = df['pl_masse']
df.loc[df['pl_masse_filled'].isna(), 'pl_masse_filled'] = df['pl_masse_ml']

df['pl_rade_filled'] = df['pl_rade']
df.loc[df['pl_rade_filled'].isna(), 'pl_rade_filled'] = df['pl_rade_ml']

print(df.head())

   rowid  planet_id  star_id  system_id  pl_orbper  pl_rade  pl_masse  \
0    1.0          3    22541          1     323.21      NaN       NaN   
1    2.0          3        2          1     326.03      NaN       NaN   
2    3.0          3    22539          1        NaN      NaN       NaN   
3    4.0          6     2227          2     516.22      NaN       NaN   
4    5.0          6    22437          2     516.22      NaN       NaN   

    pl_name  discoverymethod  disc_year  ... st_rad  st_teff  sy_dist  sy_plx  \
0  11 Com b  Radial Velocity     2007.0  ...  10.86   4841.0    93.18   10.71   
1  11 Com b  Radial Velocity     2007.0  ...  19.00   4742.0    93.18   10.71   
2  11 Com b  Radial Velocity     2007.0  ...    NaN      NaN    93.18   10.71   
3  11 UMi b  Radial Velocity     2009.0  ...  10.22   4580.0   125.32    7.95   
4  11 UMi b  Radial Velocity     2009.0  ...   1.45   6397.0   125.32    7.95   

   pl_masse_ml  pl_masse_source  pl_rade_ml pl_rade_source  pl_masse_fille

In [25]:
## Export full DataFrame to CSV for analysis in another notebook
df.to_csv("exoplanets_merged_ml.csv", index=False)

print("ML-imputed merged dataset exported successfully!")

ML-imputed merged dataset exported successfully!
